### 데이터 준비

#### 데이터 불러오기

In [1]:
import pandas as pd
stock_info = pd.read_excel("../../데이터/211104_시가총액.xlsx")
KOSPI_list = stock_info.loc[stock_info["시장구분"] == "KOSPI", "종목명"].tolist()
KOSDAQ_list = stock_info.loc[stock_info["시장구분"] == "KOSDAQ", "종목명"].tolist()
TOP100_list = stock_info.sort_values(by = "시가총액", ascending = False)['종목명'].iloc[:100].tolist()

In [2]:
import os
sp_data_dict = dict()
for file_name in os.listdir("../../데이터/주가데이터"):
    stock_name = file_name.replace('.csv', '')
    sp_data = pd.read_csv("../../데이터/주가데이터/" + file_name, 
                           usecols = ["Date", "Close"], parse_dates = ['Date'])
    sp_data_dict[stock_name] = sp_data

#### 정배열 및 역배열 컬럼 생성

In [3]:
for stock_name in sp_data_dict.keys():
    sp_data = sp_data_dict[stock_name]
    sp_data["5일선"] = sp_data["Close"].rolling(5).mean()
    sp_data["20일선"] = sp_data["Close"].rolling(20).mean()
    sp_data["60일선"] = sp_data["Close"].rolling(60).mean()
    sp_data["120일선"] = sp_data["Close"].rolling(120).mean()
    sp_data.dropna(inplace = True)

    sp_data["정배열"] = (sp_data["5일선"] >= sp_data["20일선"])\
                          & (sp_data["20일선"] >= sp_data["60일선"])\
                          & (sp_data["60일선"] >= sp_data["120일선"])

    sp_data["역배열"] = (sp_data["5일선"] <= sp_data["20일선"])\
                          & (sp_data["20일선"] <= sp_data["60일선"])\
                          & (sp_data["60일선"] <= sp_data["120일선"])

### 정배열 구간에서의 기대 수익 계산

#### 매매 전략 구현

In [4]:
import numpy as np
def calc_ror_in_forward(sp_data):
    ror_list = []
    buy_point_list = sp_data["정배열"].values[1:] > sp_data["정배열"].values[:-1]
    buy_point_list = np.insert(buy_point_list, 0, False)
    buy_point_list = sp_data.index[buy_point_list]
    sell_point_list = sp_data["정배열"].values[:-1] > sp_data["정배열"].values[1:]
    sell_point_list = np.insert(sell_point_list, 0, False)
    sell_point_list = sp_data.index[sell_point_list]

    for buy_point in buy_point_list:
        if (sum(buy_point<sell_point_list) > 0) and (buy_point+1 <= sp_data.index[-1]):
            buy_price = sp_data.loc[buy_point + 1, "Close"]
            sell_point = sell_point_list[sell_point_list > buy_point][0] + 1
            if sell_point <= sp_data.index[-1]:
                sell_price = sp_data.loc[sell_point, "Close"]
                ror = (sell_price - buy_price) / buy_price * 100
                ror_list.append(ror)
        else:
            break
    return ror_list

#### 전체 종목

In [5]:
total_ror_list = []
for stock_name in sp_data_dict.keys():
    sp_data = sp_data_dict[stock_name]
    ror_list = calc_ror_in_forward(sp_data)
    total_ror_list += ror_list
display(pd.Series(total_ror_list).describe().round(2))

count    57347.00
mean         1.00
std         18.74
min        -89.05
25%         -5.75
50%         -1.74
75%          2.25
max        701.56
dtype: float64

#### 코스피 종목

In [6]:
total_ror_list = []
for stock_name in sp_data_dict.keys() & set(KOSPI_list):
    stock_data = sp_data_dict[stock_name]
    ror_list = calc_ror_in_forward(stock_data)
    total_ror_list += ror_list
display(pd.Series(total_ror_list).describe().round(2))

count    22825.00
mean         1.00
std         15.87
min        -50.95
25%         -4.83
50%         -1.40
75%          2.21
max        348.42
dtype: float64

#### 코스닥 종목

In [7]:
total_ror_list = []
for stock_name in sp_data_dict.keys() & set(KOSDAQ_list):
    stock_data = sp_data_dict[stock_name]
    ror_list = calc_ror_in_forward(stock_data)
    total_ror_list += ror_list
display(pd.Series(total_ror_list).describe().round(2))

count    34336.00
mean         0.99
std         20.39
min        -89.05
25%         -6.39
50%         -2.01
75%          2.27
max        701.56
dtype: float64

#### 우량 종목

In [8]:
total_ror_list = []
for stock_name in sp_data_dict.keys() & set(TOP100_list):
    stock_data = sp_data_dict[stock_name]
    ror_list = calc_ror_in_forward(stock_data)
    total_ror_list += ror_list
display(pd.Series(total_ror_list).describe().round(2))

count    2707.00
mean        1.25
std        12.97
min       -25.41
25%        -4.16
50%        -1.27
75%         2.30
max       198.34
dtype: float64

### 정배열 시작 – 역배열 시작 구간에서의 기대 수익 계산

#### 매매 전략 구현

In [9]:
def calc_ror_in_for_rev(sp_data):
    ror_list = []
    buy_point_list = sp_data["정배열"].values[1:] > sp_data["정배열"].values[:-1]
    buy_point_list = np.insert(buy_point_list, 0, False)
    buy_point_list = sp_data.index[buy_point_list]
    sell_point_list = sp_data["역배열"].values[1:] > sp_data["역배열"].values[:-1]
    sell_point_list = np.insert(sell_point_list, 0, False)
    sell_point_list = sp_data.index[sell_point_list]

    for buy_point in buy_point_list:
        if (sum(buy_point<sell_point_list) > 0) and (buy_point+1<=sp_data.index[-1]):
            buy_price = sp_data.loc[buy_point + 1, "Close"]
            sell_point = sell_point_list[sell_point_list > buy_point][0] + 1
            if sell_point <= sp_data.index[-1]:
                sell_price = sp_data.loc[sell_point, "Close"]
                ror = (sell_price - buy_price) / buy_price * 100
                ror_list.append(ror)
        else:
            break
    return ror_list

#### 전체 종목

In [10]:
total_ror_list = []
for stock_name in sp_data_dict.keys():
    sp_data = sp_data_dict[stock_name]
    ror_list = calc_ror_in_for_rev(sp_data)
    total_ror_list += ror_list
display(pd.Series(total_ror_list).describe().round(2))

count    52251.00
mean         4.32
std         57.97
min        -95.39
25%        -17.61
50%         -8.34
75%          6.33
max       2537.21
dtype: float64

#### 코스피 종목

In [11]:
total_ror_list = []
for stock_name in sp_data_dict.keys() & set(KOSPI_list):
    stock_data = sp_data_dict[stock_name]
    ror_list = calc_ror_in_for_rev(stock_data)
    total_ror_list += ror_list
display(pd.Series(total_ror_list).describe().round(2))

count    20681.00
mean         5.25
std         55.75
min        -78.44
25%        -15.19
50%         -6.92
75%          6.89
max       1190.46
dtype: float64

#### 코스닥 종목

In [12]:
total_ror_list = []
for stock_name in sp_data_dict.keys() & set(KOSDAQ_list):
    stock_data = sp_data_dict[stock_name]
    ror_list = calc_ror_in_for_rev(stock_data)
    total_ror_list += ror_list
display(pd.Series(total_ror_list).describe().round(2))

count    31402.00
mean         3.72
std         59.44
min        -95.39
25%        -19.27
50%         -9.45
75%          5.97
max       2537.21
dtype: float64

#### 우량 종목

In [13]:
total_ror_list = []
for stock_name in sp_data_dict.keys() & set(TOP100_list):
    stock_data = sp_data_dict[stock_name]
    ror_list = calc_ror_in_for_rev(stock_data)
    total_ror_list += ror_list
display(pd.Series(total_ror_list).describe().round(2))

count    2455.00
mean        9.25
std        60.44
min       -51.08
25%       -12.59
50%        -5.83
75%         9.13
max       782.85
dtype: float64